In [1]:
import sys
sys.path.insert(0, '../../')
from time import sleep

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyplr import stlab

In [2]:
d = stlab.STLAB(username='admin', identity=1, password='83e47941d9e930f6')

STLAB device setup complete...


In [20]:
def interp_spectra(spectra):
    '''
    This function needs generalising.

    Parameters
    ----------
    spectra : TYPE
        DESCRIPTION.

    Returns
    -------
    intp_tbl : TYPE
        DESCRIPTION.

    '''
    #tbl = spectra.unstack(level=0)
    #tbl.columns = [val[1] for val in tbl.columns]
    
    intp_tbl = pd.DataFrame()
    for led, df in spectra.groupby(['led']):
        intensities = df.index.get_level_values('intensity')
        new_intensities = np.linspace(intensities.min(), intensities.max(), 4096)
        new_intensities = new_intensities.astype('int')
        df.reset_index(inplace=True, drop=True)
        #df.columns = range(0, df.shape[1])
        df.index = df.index * 63
        n = df.reindex(new_intensities).interpolate(method='linear')
        n['intensity'] = n.index
        n['led'] = led
        intp_tbl = intp_tbl.append(n)
    intp_tbl.set_index(['led','intensity'], inplace=True)
    return intp_tbl

In [26]:
cc = stlab.CalibrationContext('../data/corrected_oo_spectra.csv')
LED3_max = 1800 # comfortable maximal melanopsin stim
r_matches = {}
for led in [1,3,5,9]:
    match = cc.match(target_led=3, target_led_intensity=LED3_max, match_led=led, match_type='radiance')
    r_matches[led] = match

idxs = [(led,intensity[1]) for led, intensity in r_matches.items()]
stim_info = cc.aopic.loc[idxs]
stim_info['lux'] = cc.lux.loc[idxs]
stim_info['radiance'] = cc.lkp.sum(axis=1)
stim_info

,,S,M,L,Rods,Mel,lux,radiance
led,intensity,,,,,,,
1,1559,3.634638,0.590597,0.421508,1.636942,1.978634,171.666063,4.413626
3,1800,2.095418,1.463381,0.948496,3.058520,3.574523,474.484999,4.413422
5,2039,0.151460,3.603402,3.025069,3.355829,2.475810,2125.065087,4.413977
9,1322,0.052944,0.131397,0.676928,0.069964,0.063986,341.689773,4.413424


In [30]:
df.to_dict()

{'S': 0.05294361528531646,
 'M': 0.1313966482401933,
 'L': 0.6769275214135744,
 'Rods': 0.06996414740924106,
 'Mel': 0.06398552865688464,
 'lux': 341.6897728123068,
 'radiance': 4.413423888773993}

In [36]:
from scipy.signal import chirp

for (led, intensity), info in stim_info.iterrows():

    # stimulus params
    videoname = 'chirp_{}'.format(led)
    Fs = 100
    mintensity = 0
    maxtensity = intensity

    # stimulus params
    background  = maxtensity/2
    amplitude   = maxtensity/2
    duration = 20
    n = 2000
    t = np.linspace(0, duration, n, endpoint=False)
    f0 = 0.01
    f1 = 1

    # define frequency modulation
    chp = chirp(t, f0, duration, f1, method='linear', phi=270)
    chp = stlab.modulate_intensity_amplitude(chp, background, amplitude)
    #fig, ax = plt.subplots(figsize=(12,4))
    #ax.plot(chp)
    #ax.set_ylabel("Intensity")
    #ax.set_xlabel("Time")

    # define amplitude modulation
    time  = stlab.get_time_vector(duration)
    sm = stlab.sinusoid_modulation(.5, duration, Fs)
    ivals = stlab.modulate_intensity_amplitude(sm, background, np.linspace(0,amplitude,duration*Fs))
    #fig, ax = plt.subplots(figsize=(12,4))
    #ax.plot(ivals)
    #ax.set_ylabel("Intensity")
    #ax.set_xlabel("Time");

    duration=60
    time = stlab.get_time_vector(duration)
    stim = np.zeros(duration*Fs)

    # assemble time course
    stim[0:100]     = maxtensity # 1s pulse
    stim[800:900]   = background # step
    stim[900:2900]  = chp  # chirp
    stim[2900:3100] = background
    stim[3100:5100] = ivals
    stim[5100:5300] = background
    stim = stim.astype('int')

    # plot
    #fig, ax = plt.subplots(figsize=(16,4))
    #ax.plot(stim)
    #ax.set_ylabel("Intensity")
    #ax.set_xlabel("Time")

    # make video file
    cols = stlab.get_video_cols()
    zs = np.zeros(duration*Fs)
    zs = zs.astype('int')
    data = [zs for x in range(10)]
    data[led] = stim

    #data = [stim for x in range(10)]
    data.insert(0, time)
    df = pd.DataFrame(data, cols).T
    setting = {'led':led,'max_intensity':intensity}
    info = {**info.to_dict(), **setting}
    stlab.make_video_file(df, videoname, **info)

"chirp_1.dsf" saved in the current working directory.
"chirp_3.dsf" saved in the current working directory.
"chirp_5.dsf" saved in the current working directory.
"chirp_9.dsf" saved in the current working directory.


In [37]:
stlab.video_file_to_dict('chirp_1.dsf')

{'header': {'version': 1,
  'model': 'VEGA10',
  'channels': 10,
  'spectracount': 6000,
  'transitionsCount': 6000,
  'fluxReference': 0,
  'repeats': 1},
 'metadata': {'creation_time': '2020-12-08 12:45:24.072245',
  'creator': 'jtm',
  'S': 3.6346380030214767,
  'M': 0.5905972300668568,
  'L': 0.42150846837789235,
  'Rods': 1.6369418924190529,
  'Mel': 1.978633506622812,
  'lux': 171.66606306716082,
  'radiance': 4.413625599638963,
  'led': 1,
  'max_intensity': 1559},
 'spectra': [[0, 1559, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1559, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1559, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1559, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1559, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1559, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1559, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1559, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1559, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1559, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1559, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1559, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1559, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1559, 0, 0, 0, 0, 0, 0, 0

In [8]:
import glob
from time import sleep

for chp in glob.glob('chirp_*.dsf'):
    d.load_video_file(chp)
    d.play_video_file()
    sleep(60)

video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...
video file loaded...
playing video file...


In [28]:
glob.glob('chirp_*.dsf')

['chirp_4.dsf',
 'chirp_5.dsf',
 'chirp_7.dsf',
 'chirp_6.dsf',
 'chirp_2.dsf',
 'chirp_3.dsf',
 'chirp_1.dsf',
 'chirp_0.dsf',
 'chirp_8.dsf',
 'chirp_9.dsf']